In [2]:
import tensorflow as tf
import numpy as np

## Follow the official "get started" 

References:

- https://www.tensorflow.org/get_started/get_started

In [33]:
# create scalar constant tensors
# scalars are rank 0 tensor

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0, dtype=tf.float32)
print(node1)
print(node2)

# shape is empty because they are scalars
print(node1.get_shape())
print(node2.get_shape())


Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
()
()


In [43]:
# tensors must be evaluated within a session
# Session.run() takes tensors and evaluate them
with tf.Session() as sess:
    print(sess.run([node1, node2]))
    
# tf.rank() returns the rank of an object as a tensor
# rank zero means a scalar
with tf.Session() as sess:
    print(sess.run([tf.rank(node1), tf.rank(node2)])) 

[3.0, 4.0]
[0, 0]


In [44]:
# tensors support a variety of operations
# tf.add(x, y) creates a tensor that adds up two tensors x and y
node3 = tf.add(node1, node2)
print(node3)
with tf.Session() as sess:
    print(sess.run(node3))

Tensor("Add_15:0", shape=(), dtype=float32)
7.0


In [42]:
# placeholders defines a tensor for which values are supplied later 
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.add(a, b)
with tf.Session() as sess:
    print(sess.run(c, {a: -1.0, b: 3}))

# vector values can be fed
with tf.Session() as sess:
    print(sess.run(c, {a: [-1.0, 10.0], b: [3.0, 4.0]}))

2.0
[  2.  14.]


In [ ]:
# Variables are trainable parameters
W = tf.Variable([0.3], dtype=tf.float32, name="weight")
b = tf.Variable([-0.3], dtype=tf.float32, name="bias")
x = tf.placeholder(tf.float32)
yhat = tf.add(tf.multiply(W, x), b, name="yhat")

print(W)
print(b)
print(x)
print(yhat)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(yhat, {x: [1, 2, 3, 4]}))
    

In [62]:
# compare and train with true values
y = tf.placeholder(tf.float32)
loss = tf.reduce_mean(tf.square(tf.subtract(y, yhat)))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]}))

# set values mannually to W and b to improve the fit
# `tf.assign()` defines an operation to change variable values
# note that this does not happen until given to `run`
fixW = tf.assign(W, [-1.0])
fixb = tf.assign(b, [1.0])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run([fixW, fixb])
    print(sess.run(loss, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]}))


5.735
0.015


In [77]:
# supervised training to find better variables values
# `train` here defines an operation to reduces the loss by
# the gradient descent
# again, this shall be given to `run` to invoke it
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # update the variables for multiple times
    for i in range(1000):
        sess.run(train, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]})
    
    print(sess.run(loss, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]}))
    print(sess.run([W, b]))
    # or print the values by
    print(W.eval(), b.eval())

0.0135716
[array([-0.97275525], dtype=float32), array([ 0.96989703], dtype=float32)]
[-0.97275525] [ 0.96989703]


## Computation graph example

References: 

- Saito, K (2016). Deep Learning from Scratch, O'Reilly Japan.

In [107]:
# Fig 5-17. p.140
num_apple    = tf.placeholder(tf.float32)
price_apple  = tf.Variable([100.0], tf.float32)
num_orange   = tf.placeholder(tf.float32)
price_orange = tf.Variable([150.0], tf.float32)
taxrate      = tf.Variable([0.1], tf.float32)

pretax       = tf.add(tf.multiply(num_apple,  price_apple),
                      tf.multiply(num_orange, price_orange))
expenditure  = tf.add(pretax, tf.multiply(pretax, taxrate))
print(num_apple)
print(price_apple)
print(num_orange)
print(price_orange)
print(taxrate)
print(pretax)
print(expenditure)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(expenditure, {num_apple: 2, num_orange: 3}))

Tensor("Placeholder_53:0", dtype=float32)
<tf.Variable 'Variable_55:0' shape=(1,) dtype=float32_ref>
Tensor("Placeholder_54:0", dtype=float32)
<tf.Variable 'Variable_56:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'Variable_57:0' shape=(1,) dtype=float32_ref>
Tensor("Add_41:0", dtype=float32)
Tensor("Add_42:0", dtype=float32)
[ 715.]


In [108]:
# compute the grandients
# tf.gradients(y, x) to get dy/dx

grads = tf.gradients(
    expenditure, 
    [num_apple, price_apple, num_orange, price_orange, taxrate]
)
print(grads)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(grads, {num_apple: 2, num_orange: 3}))

[<tf.Tensor 'gradients_23/Mul_48_grad/Reshape:0' shape=<unknown> dtype=float32>, <tf.Tensor 'gradients_23/Mul_48_grad/Reshape_1:0' shape=(1,) dtype=float32>, <tf.Tensor 'gradients_23/Mul_49_grad/Reshape:0' shape=<unknown> dtype=float32>, <tf.Tensor 'gradients_23/Mul_49_grad/Reshape_1:0' shape=(1,) dtype=float32>, <tf.Tensor 'gradients_23/Mul_50_grad/Reshape_1:0' shape=(1,) dtype=float32>]
[110.0, array([ 2.20000005], dtype=float32), 165.0, array([ 3.30000019], dtype=float32), array([ 650.], dtype=float32)]


## Minimizing quadratic functions

In [59]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.placeholder(tf.float32)
x = tf.Variable(0.0, tf.float32)

y = a*x*x + b*x + c

print(a)
print(b)
print(c)
print(x)
print(y)

# find the mimimum
optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(y)

data = {a: 0.5, b: 5.0, c: 3.0}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # update the variables for multiple times
    for i in range(100):
        sess.run(train, data)
    
    print(sess.run([x, y], data))

# expected: x = -b/(2a)
x_star = -data[b]/data[a]/2
y_star = data[a]*x_star*x_star + data[b]*x_star + data[c]
print("expected: x = ", x_star, "y = ", y_star)

Tensor("Placeholder_7:0", dtype=float32)
Tensor("Placeholder_8:0", dtype=float32)
Tensor("Placeholder_9:0", dtype=float32)
<tf.Variable 'Variable:0' shape=() dtype=float32_ref>
Tensor("add_1:0", dtype=float32)
[-4.999867, -9.499999]
expected: x =  -5.0 y =  -9.5


## Estimating and maximizing a production function

We will generate a dummy data of $(K, L, Y)$ from $Y = K^\alpha L^{1-\alpha}$ and estimate $Y = f(K, L)$ by neural net.  Then, we will maximize: $f(K, L)$ subject to $r K + w L = 1$.

This optimization problem has an analytic solution: 

$K^* = \frac{\alpha}{r}, L^* = \frac{1-\alpha}{w}$ 

We expect the optimization results are close to these.

In [48]:
def generate_cobb_douglas(N, alpha):
    """
    Generate (K, L, Y) such that Y = K^alpha L^(1-alpha) 
    
    K and L are generated from independent exponential distribution with mean 1
    
    N       : number of observation
    alpha   : power to K
    sigma   : standard deviation of epsilon 
    """
    K = np.random.exponential(1, (N,))
    L = np.random.exponential(1, (N,))
    Y = np.power(K, alpha) * np.power(L, 1-alpha)
    
    return K, L, Y

Ks, Ls, Ys = generate_cobb_douglas(100, 0.3)

# this should be very close to zero
np.max(np.abs(np.log(Ys) - 0.3 * np.log(Ks) - 0.7 * np.log(Ls)))

4.4408920985006262e-16

In [146]:
# 3 layer full-connected neural net
X  = tf.placeholder(tf.float32, shape=[None, 2])

W1 = tf.Variable(tf.truncated_normal([2, 10], stddev=0.1), dtype=tf.float32)
b1 = tf.Variable(tf.truncated_normal([10], stddev=0.1), dtype=tf.float32)
A1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([10, 10], stddev=0.1), dtype=tf.float32)
b2 = tf.Variable(tf.truncated_normal([10], stddev=0.1), dtype=tf.float32)
A2 = tf.nn.relu(tf.matmul(A1, W2) + b2)

W3 = tf.Variable(tf.truncated_normal([10, 10], stddev=0.1), dtype=tf.float32)
b3 = tf.Variable(tf.truncated_normal([10], stddev=0.1), dtype=tf.float32)
O  = tf.nn.relu(tf.matmul(A3, W3) + b3)

# squared error 
Y = tf.placeholder(tf.float32)    
loss = tf.reduce_mean(tf.square(O - Y))

# optimizer
train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [147]:
Ks, Ls, Ys = generate_cobb_douglas(100, 0.3)
in_x = np.vstack((Ks, Ls)).transpose()
in_y = Ys.reshape(100, 1)

data = {X: in_x, Y: in_y}

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        sess.run(train, feed_dict=data)
        print(sess.run(loss))
    print(sess.run(loss))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_39' with dtype float and shape [?,2]
	 [[Node: Placeholder_39 = Placeholder[dtype=DT_FLOAT, shape=[?,2], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_39', defined at:
  File "/home/kota/miniconda3/envs/nnet/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kota/miniconda3/envs/nnet/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kota/miniconda3/envs/nnet/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kota/miniconda3/envs/nnet/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kota/miniconda3/envs/nnet/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/kota/miniconda3/envs/nnet/

In [142]:
a = tf.placeholder(tf.float32, shape=[None, 2], name="a")
s = tf.reduce_mean(a)
d = {a: np.vstack((Ks, Ls)).transpose()}
with tf.Session() as sess:
    print(sess.run(s, d))

1.04032


In [119]:
data

{<tf.Tensor 'x:0' shape=(?, 2) dtype=float32>: array([[ 1.42013198, -0.33339645],
        [-0.27852431,  0.41698799],
        [-0.18999681,  1.57584897],
        [-1.30269705, -0.11279611],
        [-1.27970328,  1.22636823],
        [-1.08295306, -1.06615658],
        [ 0.67871907,  2.26270686],
        [ 1.11795212, -0.03750866],
        [-3.19198343, -0.7514159 ],
        [-1.76418003, -0.71878941],
        [-0.24390842,  0.23197449],
        [-0.07816966,  0.09565085],
        [ 0.29544513, -1.12167588],
        [-0.89361596, -0.8950144 ],
        [ 0.32034957,  0.39183769],
        [ 0.611329  , -0.31773639],
        [-0.05108229,  0.58413104],
        [ 2.15245247, -0.3263996 ],
        [ 0.30240692, -0.05640359],
        [ 1.63882231,  0.37254891],
        [-0.5991681 , -1.20108367],
        [ 0.70241323, -0.44715967],
        [-0.185286  , -0.33194316],
        [-0.60643069, -0.65566619],
        [-0.46578149,  0.12067983],
        [-0.32817437,  0.5600664 ],
        [-0.837259

In [101]:
data[Y].shape

(100, 1)

In [103]:
import tensorflow as tf
import numpy as np

x = tf.placeholder(tf.float32, shape=[None, 2])
y_ = tf.placeholder(tf.float32, shape=[None, 2])
loss = tf.reduce_sum(tf.abs(tf.subtract(x, y_)))#Function chosen arbitrarily
input_x=np.random.randn(100, 2)#Random generation of variable x
input_y=np.random.randn(100, 2)#Random generation of variable y

with tf.Session() as sess:
    print(sess.run(loss, feed_dict={x: input_x, y_: input_y}))

233.461


In [107]:
input_x.shape
input_y.shape

(100, 2)